In [1]:
import pandas as pd
import pathlib
from papermill import execute_notebook

In [2]:
cell_class = ['Inh']
output_dir = '/home/hanliu/project/mouse_rostral_brain/study/ClustersEnsemble/InhSubtypeEnsemble'
pathlib.Path(output_dir).mkdir(exist_ok=True)

## Cell Meta

In [3]:
cell_tidy_data = pd.read_msgpack(
    '/home/hanliu/project/mouse_rostral_brain/study/ClusteringSummary/Summary/TotalClusteringResults.msg'
)
cell_tidy_data['MajorType'].unique()

/home/hanliu/miniconda3/envs/clustering/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3326: FutureWarning: The read_msgpack is deprecated and will be removed in a future version.
It is recommended to use pyarrow for on-the-wire transmission of pandas objects.
  exec(code_obj, self.user_global_ns, self.user_ns)


array(['MGE-Sst', 'CA3', 'CA1', 'CA3-St18', 'Unc5c', 'Gfra1', 'ODC', 'PC',
       'ANP', 'IT-L5', 'NP-L6', 'CGE-Lamp5', 'CT-L6', 'IG-CA2', 'DG-po',
       'DG', 'CGE-Vip', 'OPC', 'ASC', 'MGC', 'PAL-Inh', 'PT-L5',
       'MGE-Pvalb', 'VLMC', 'EC', 'VLMC-Pia', 'OLF', 'MSN-D2', 'L6b',
       'IT-L6', 'IT-L23', 'IT-L4', 'OLF-Exc', 'CLA', 'Foxp2', 'MSN-D1',
       'LSX-Inh', 'D1L-Fstl4', 'EP', 'D1L-PAL', 'Chd7'], dtype=object)

## Select significant subtype of CP ACB

In [4]:
use_clusters = sorted(cell_tidy_data[cell_tidy_data['CellClass'].isin(
    cell_class)]['SubType'].unique())
use_clusters = [i for i in use_clusters if 'Outlier' not in i]
use_clusters

['CGE-Lamp5 Dock5',
 'CGE-Lamp5 Grid1',
 'CGE-Lamp5 Grk5',
 'CGE-Lamp5 Nrxn3',
 'CGE-Lamp5 Sorcs1',
 'CGE-Vip Ccser1',
 'CGE-Vip Clstn2',
 'CGE-Vip Fstl4',
 'CGE-Vip Galnt17',
 'CGE-Vip Grm8',
 'CGE-Vip Ntng1',
 'CGE-Vip Ptprm',
 'CGE-Vip Robo1',
 'Chd7 Kcnc2',
 'Chd7 Megf11',
 'Chd7 Trpc7',
 'D1L-Fstl4 Cadm1',
 'D1L-Fstl4 Crim1',
 'D1L-Fstl4 Grm3',
 'D1L-Fstl4 Sipa1l2',
 'D1L-Fstl4 Trps1',
 'D1L-PAL Flrt2',
 'D1L-PAL Plcxd3',
 'Foxp2 Dchs2',
 'Foxp2 Homer2',
 'Foxp2 Inpp4b',
 'Foxp2 Trpc7',
 'LSX-Inh Cacna1i',
 'LSX-Inh Dock10',
 'LSX-Inh Enox1',
 'LSX-Inh Foxp2',
 'LSX-Inh Lats2',
 'LSX-Inh Nxph1',
 'LSX-Inh Zeb2',
 'MGE-Pvalb Cacna1i',
 'MGE-Pvalb Cnih3',
 'MGE-Pvalb Entpd3',
 'MGE-Pvalb Gfra2',
 'MGE-Pvalb Ptprk',
 'MGE-Pvalb Sema5a',
 'MGE-Pvalb Thsd7a',
 'MGE-Sst Bmper',
 'MGE-Sst Chodl',
 'MGE-Sst Dock4',
 'MGE-Sst Etv1',
 'MGE-Sst Frmd6',
 'MGE-Sst Kcnip4',
 'MGE-Sst Ptpre',
 'MGE-Sst Rerg',
 'MGE-Sst Rxra',
 'MGE-Sst Ubtd1',
 'MGE-Sst Unc5b',
 'MSN-D1 Hrh1',
 'MSN-D1 Khdrbs3',

## Step1 Select DMG

In [5]:
parameters = dict(auroc_cutoff=0.85, use_clusters=use_clusters)

input_path = 'related_dmgs.ipynb'
output_path = f'{output_dir}/related_dmgs.ipynb'

execute_notebook(str(input_path),
                 str(output_path),
                 parameters=parameters,
                 engine_name=None,
                 prepare_only=False,
                 kernel_name=None,
                 progress_bar=True,
                 log_output=False,
                 start_timeout=60,
                 report_mode=False,
                 cwd=output_dir)
pass

## Step2 Select DMR

In [6]:
parameters = dict(
    use_clusters=[i.replace(' ', '_') for i in use_clusters],
    hypo_dmr_hits_path=
    '/home/hanliu/project/mouse_rostral_brain/DMR/SubType/Total/TotalHits.HypoDMR.h5ad'
)

input_path = 'related_dmrs.ipynb'
output_path = f'{output_dir}/related_dmrs.ipynb'

execute_notebook(str(input_path),
                 str(output_path),
                 parameters=parameters,
                 engine_name=None,
                 prepare_only=False,
                 kernel_name=None,
                 progress_bar=True,
                 log_output=False,
                 start_timeout=60,
                 report_mode=False,
                 cwd=output_dir)
pass

## Step3 Select corr

In [7]:
parameters = dict(corr_cutoff=0.5, distance_cutoff=500000)

input_path = 'related_corr.ipynb'
output_path = f'{output_dir}/related_corr.ipynb'

execute_notebook(str(input_path),
                 str(output_path),
                 parameters=parameters,
                 engine_name=None,
                 prepare_only=False,
                 kernel_name=None,
                 progress_bar=True,
                 log_output=False,
                 start_timeout=60,
                 report_mode=False,
                 cwd=output_dir)
pass

## Step4 intersect everything

In [8]:
parameters = dict(delta_dmr_rate_cutoff = 0.3)

input_path = 'intersect_everything.ipynb'
output_path = f'{output_dir}/intersect_everything.ipynb'

execute_notebook(str(input_path),
                 str(output_path),
                 parameters=parameters,
                 engine_name=None,
                 prepare_only=False,
                 kernel_name=None,
                 progress_bar=True,
                 log_output=False,
                 start_timeout=60,
                 report_mode=False,
                 cwd=output_dir)
pass

## Motif enrichment

In [ ]:
motif_enrichment_dir = pathlib.Path(output_dir) / 'MotifEnrichment'
motif_enrichment_dir.mkdir(exist_ok=True)

In [ ]:
_use_clusters = [i.replace(' ', '_') for i in use_clusters]
for cluster in _use_clusters:
    print(cluster)
    parameters = dict(cluster=cluster,
                      use_clusters=_use_clusters,
                      or_cutoff=1.6,
                      neg_lgp_cutoff=10,
                      mask_quantile_to_max = 0.8)

    input_path = 'MotifEnrichment.ipynb'
    output_path = f'{motif_enrichment_dir}/MotifEnrichment.{cluster}.ipynb'

    execute_notebook(str(input_path),
                     str(output_path),
                     parameters=parameters,
                     engine_name=None,
                     prepare_only=False,
                     kernel_name=None,
                     progress_bar=True,
                     log_output=False,
                     start_timeout=60,
                     report_mode=False,
                     cwd=str(motif_enrichment_dir))
    

CA1_Ak5



CA1_Chrm3



CA1_Kif26a



CA1_Lingo2



CA1_Ptprg



CA3_Cadm2



CA3_Efnb2



CA3-St18_Epha5



CA3-St18_Nuak1


### Aggregate motif enrichment

In [ ]:
parameters = dict(
    oddsratio_cutoff=1.8,
    lgp_cutoff=10,
    motif_enrichment_dir=str(motif_enrichment_dir),
    tf_class_level='SubFamily',
)

input_path = 'aggregate_motif_enrichment.ipynb'
output_path = f'{motif_enrichment_dir}/aggregate_motif_enrichment.ipynb'

execute_notebook(str(input_path),
                 str(output_path),
                 parameters=parameters,
                 engine_name=None,
                 prepare_only=False,
                 kernel_name=None,
                 progress_bar=True,
                 log_output=False,
                 start_timeout=60,
                 report_mode=False,
                 cwd=str(motif_enrichment_dir))
pass